**WRITER'S IDENTITY**
- AZIMIL GANI ALAM
- Ph.D Student
- Dept. Energy & Process Tech - NTNU


**NOTE :**
- This Python File runs in GoogleColab
- This work uses **NILU - IS Data : Questionnaire Results**
- we want to filterize data results from CSV models become process-ready for Machine Learning process

**RULES**
1. Run Library Import
2. Run 'Default of Names
3. Expand Schools Tab
    - Run Identity
    - Run Data Caller
    - Run K
4. Expand Classrooms
    - Run Klasseroms one by one.

to check the data cleaning result, run 'reviewer'. there is no cyclic command.


# **Library Import**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from datetime import datetime, timedelta
import pytz

# Default  Names & Setting

In [2]:
col_header = ['title', 'sessionid', 'date', 'time' ]
response_counting = ['responses']
is_col_main  = ['feel_air','feel_temp','feel_bright','feel_health','feel_noise']
is_col_air   = ['air_smell','air_heavy','air_dry','air_dust','air_electshock']
is_col_temp  = ['temp_coldhot','temp_draw','temp_coldfloor','temp_heatsun','temp_heater' ]
is_col_light = ['bright_sun','bright_lamp_hi','bright_lamp_low']
is_col_feel  = ['health_head','health_cough','health_tired','health_dryskin']
columns_formation   = col_header + is_col_main + is_col_air + is_col_temp + is_col_light + is_col_feel

school_col_name   = 'school_id'
room_col_name     = 'room_id'
date_name         = 'date'
time_name         = 'time'
manuf_col_name    = 'manuf_id'
instr_col_name    = 'instr_id'
serial_col_name   = 'serial_id'

#Give a name for new column in purpose to count how many response / hour or response / day
response_counting = 'responses'

#Folder naming directory
platform_directory = 'C:/Users/azimilga'
folder_raw = 'data-raw'
folder_ready = 'data-ready'

folder_project = 'project'
folder_export = 'export'
folder_is = 'niluapp'

all_columns_set = {'HVORDAN ER FORMEN DIN NÅ?' : 'feel_health', 
                   'LUFTEN I ROMMET NÅ': 'feel_air', 
                   'LYS I ROMMET NÅ': 'feel_bright',
                   'STØY I ROMMET NÅ' : 'feel_noise',
                   'TEMPERATUR I ROMMET NÅ': 'feel_temp',
                   
                   'VOND LUKT' : 'air_smell' , 
                   'DÅRLIG/TUNG LUFT' : 'air_heavy', 
                   'TØRR LUFT' : 'air_dry', 
                   'STØV OG SKITT' : 'air_dust',
                   'ELEKTRISK STØT' : 'air_electshock',
                   
                   'KALDT ELLER VARMT' : 'temp_coldhot',
                   'TREKK KALD LUFT' : 'temp_draw',
                   'FOR KALDT PÅ GULVET' : 'temp_coldfloor',
                   'FOR MYE VARME FRA SOLSKINN' : 'temp_heatsun',
                   'FOR MYE VARME FRA OVNER' : 'temp_heater' ,
                   
                   'SKARPT LYS FRA SOLA' : 'bright_sun',
                   'SKARPT LYS TAKLAMPER' : 'bright_lamp_hi', 
                   'SVAKT LYS TAKLAMPER' : 'bright_lamp_low',
                   
                   'HODEPINE' : 'health_head',
                   'HOSTE/SNØRRETE' : 'health_cough',
                   'TRØTT/UKONSENTRERT' : 'health_tired',
                   'TØRRE ØYNE/HENDER' : 'health_dryskin'}


In [3]:
#filtering rows data regarding to desired specific time
start_time = '00:00:00'
end_time   = '23:59:59'
spec_date  = pd.to_datetime('today').strftime("%Y-%m-%d")
#name_date = pd.to_datetime('today').strftime("%y%m%d")
date_format_show =  '%Y-%m-%d'
time_format_show =  '%H:%M'

#filtering rows data regarding to desired specific time. We neglect data if there is a student did not answer anything. for example, from 808 responses, become 757 responses.
min = 0

def fill_temp_coldhot(row):
    if pd.isna(row['temp_coldhot']):
        if row['feel_temp'] <= 2:
            return 0
        else:
            return row['temp_coldhot']  # Keep NaN if feel_temp > 2
    else:
        return row['temp_coldhot']

# Bratteberg

## Identity

In [4]:
## Classroom 141, 142, 210, Øst Brakker
folder_year = '2023'
start_date =  '2023-03-08'
end_date   =  '2023-04-26'

## Classroom 141, 210, Øst Brakker
# folder_year = '2024'
# start_date = '2024-02-26'
# end_date  =  '2024-03-08'

In [5]:
school_name = 'bratteberg'
manuf_id    = 'niluapp'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_raw   + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'
export_file_directory   = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

## Data Caller

In [6]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.txt') or f.endswith('.csv')] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, ) # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
DF1 = pd.concat(dfs1, ignore_index=True)

datechange = pd.to_datetime(DF1['timestart'])
DF1['date'] = datechange.dt.strftime('%Y-%m-%d')
DF1['time'] = datechange.dt.strftime('%H:%M')

# Define the timezones (wrong time, and Europe/Oslo)
IS_timezone = pytz.timezone('Africa/Luanda')  #why africa? time recording in IS Data is little bit wrong. it has 1 hour delatation but it doesnt consider DST principle. so, applying Africa Luanda, is a trick of that
gmt_timezone = pytz.timezone('Europe/London')
oslo_timezone = pytz.timezone('Europe/Oslo')

# Adjust the datetime to the Oslo timezone and account for daylight saving time
daylight_save_convert = datechange.dt.tz_localize(IS_timezone).dt.tz_convert(gmt_timezone)

# Extract the adjusted time
DF1['time'] = daylight_save_convert.dt.strftime('%H:%M')

cols_to_move = ['title', 'surveyid', 'sessionid','date', 'time']
DF1          = DF1[ cols_to_move + [ col for col in DF1.columns if col not in cols_to_move ] ].drop(['timestart', 'timeend'], axis=1)

#this is to transform numericvalue, from 0-4 become 0-3. where 0 and 1, will be converted as 0. then 2 - 4 will be substract by 1.
#DF1['numericvalue'] = (DF1['numericvalue'].astype(int).replace({0 : 1}) - 1)


In [7]:
DF1_index = DF1.iloc[:, :5].drop(['surveyid'], axis=1).iloc[0::5,:].reset_index(level=0, drop=True)

**Fore-questions transformation**

Fore-questions is arranged as 4 questions : Air, Temperature, Noise, Health, and indoor Brightness.


In [56]:
# #we reshape the columns, from 1 column, to 5 columns, separate by different questions. select one of this following commands
# value_main = np.nan_to_num(DF1['numericvalue'].values.reshape((-5,5)), nan=0)    #NUMBER .1
# #value_main = DF1['numericvalue'].values.reshape(-5, 5)                          #NUMBER .2

# DF1_first = pd.DataFrame(value_main, columns=np.array(DF1.text.unique())).astype(int)

# #if we select NUMBER. 1, activate this command
# DF1_first.replace(0, np.nan, inplace=True)

#                                     'NOTE_THIS' :   EXPIRED. MISS INTERPRETATION AND MISSED COLUMNS' ARRANGING

In [8]:
# Pivot the dataframe so that each word becomes a column
pivot_df = DF1.pivot_table(index=DF1.groupby('text').cumcount(), columns='text', values='numericvalue', aggfunc='first')

# Reset the index to make it a proper dataframe
DF1_first = pivot_df.reset_index(drop=True)
DF1_first.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2007 entries, 0 to 2006
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   HVORDAN ER FORMEN DIN NÅ?  1989 non-null   float64
 1   LUFTEN I ROMMET NÅ         2001 non-null   float64
 2   LYS I ROMMET NÅ            1982 non-null   float64
 3   STØY I ROMMET NÅ           1989 non-null   float64
 4   TEMPERATUR I ROMMET NÅ     1995 non-null   float64
dtypes: float64(5)
memory usage: 78.5 KB


**Sub-questions arrangement**

In [9]:
DF1_air = (DF1.iloc[:, 8:13].astype(str)).replace({'False': 0, 'True': 1}).reset_index(level=0, drop=True).astype(float).round(0)
DF1_temp = (DF1.iloc[:, 13:18].astype(str)).replace({'False': 0, 'True': 1})
DF1_bright = (DF1.iloc[:, 18:21].astype(str)).replace({'False': 0, 'True': 1})
DF1_health = (DF1.iloc[:, 21:25].astype(str)).replace({'False': 0, 'True': 1})

#set how many delays you want to each variable
delay_temp = 1
delay_bright = 1 + delay_temp
delay_health = 1 + delay_bright

#set new columns for delayed input variables
DF1_tempD   = DF1_temp.iloc[delay_temp:].reset_index(drop=True).astype(float).round(0)
DF1_brightD = DF1_bright.iloc[delay_bright:].reset_index(drop=True).astype(float).round(0)
DF1_healthD = DF1_health.iloc[delay_health:].reset_index(drop=True).astype(float).round(0)

DF1_back = pd.concat([DF1_air, DF1_tempD, DF1_brightD, DF1_healthD], axis=1).iloc[0::5,:].reset_index(level=0, drop=True) #.astype(float)

#Special Action
#Cold and Warm Question has answer's range from 1 to 7. but 0 is the empty answer. we have to convert 0 as 0, but 1 to 7 become -3 to 3. note that scale-4 (means not cold nor warm) as 0.
# DF1_back['KALDT ELLER VARMT'] = DF1_back['KALDT ELLER VARMT'].astype(int).apply(lambda x: x - 4 if x > 0 else x)

**Table Assembly**

In [10]:
DF1_new = pd.concat([DF1_index, DF1_first, DF1_back], axis=1) .sort_values(['date', 'time'], ascending=True).reset_index(drop=True)
DF1_new.rename(columns=all_columns_set, inplace=True)
DF1_new['title'].unique()

array(['Bratteberg klasserom 210', 'Bratteberg Klasserom Øst brakker',
       'Bratteberg klasserom 142', 'Bratteberg klasserom 141',
       'Øyra klasserom 302-7b'], dtype=object)

In [59]:
#DF1_new[DF1_new['title']=='Øyra klasserom 302-7b']
print(input_file_directory)
print(columns_formation)

C:/Users/azimilga/project/data-raw/2024/bratteberg/niluapp/
['title', 'sessionid', 'date', 'time', 'feel_air', 'feel_temp', 'feel_bright', 'feel_health', 'feel_noise', 'air_smell', 'air_heavy', 'air_dry', 'air_dust', 'air_electshock', 'temp_coldhot', 'temp_draw', 'temp_coldfloor', 'temp_heatsun', 'temp_heater', 'bright_sun', 'bright_lamp_hi', 'bright_lamp_low', 'health_head', 'health_cough', 'health_tired', 'health_dryskin']


In [60]:
DF1_new[['feel_bright'] + is_col_light][DF1_new['feel_bright'] ==4].iloc[:,:19].describe().round(3)

,feel_bright,bright_sun,bright_lamp_hi,bright_lamp_low
count,33.0,32.000,31.000,32.000
mean,4.0,0.406,0.806,0.344
std,0.0,0.499,0.402,0.483
min,4.0,0.000,0.000,0.000
25%,4.0,0.000,1.000,0.000
50%,4.0,0.000,1.000,0.000
75%,4.0,1.000,1.000,1.000
max,4.0,1.000,1.000,1.000


## Classrooms

### Klasserom 141

In [11]:
room_id = 141
class_dir = '141'
is_data_class_name = 'Bratteberg klasserom '+class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

### Klasserom 142

In [15]:
room_id = 142
class_dir = '142'
is_data_class_name = 'Bratteberg klasserom '+class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

### Klasserom 210

In [17]:
room_id = 210
class_dir = '210'
is_data_class_name = 'Bratteberg klasserom '+class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

### Klasserom Øst Brakker

In [19]:
room_id = 'Øst brakker'
class_dir = 'Øst brakker'
is_data_class_name = 'Bratteberg Klasserom '+class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

## Reviewer

In [21]:
#testing  = ((DF2['feel_air'] + DF2['feel_temp']+DF2['feel_bright']+DF2['feel_health'] +DF2['feel_noise']))
#testing2 = ((DF2_listed['feel_air'] + DF2_listed['feel_temp']+DF2_listed['feel_bright']+DF2_listed['feel_health'] +DF2_listed['feel_noise']))
#
print ('all response        :', DF2[time_name].count() )
print ('clean response      :', DF2_listed[time_name].count())
print ('unanswered response : ', DF2[time_name].count() - DF2_listed[time_name].count())

all response        : 538
clean response      : 520
unanswered response :  18


In [ ]:
DF3 = DF2_listed
DF3['datetime'] = pd.to_datetime(DF3['date'] + ' ' + DF3['time'])
DF3[response_counting] = 1
DF3.insert(8, response_counting, DF3.pop(response_counting))
DF3_1m= DF3.set_index('datetime').resample('1min').sum().round(1)
DF3_15m= DF3.set_index('datetime').resample('15min').sum().round(1)
DF3_20m= DF3.set_index('datetime').resample('15min').sum().round(1)
DF3_30m= DF3.set_index('datetime').resample('30min').sum().round(1)
DF3_H= DF3.set_index('datetime').resample('H').sum().round(1)

In [ ]:
DF3_3H_ave= DF3.set_index('datetime').resample('3H').mean().round(2)
DF3_3H_ave['responses'] = DF3.loc[:,['responses', 'datetime']].set_index('datetime').resample('3H').sum().round(2)
DF3_3H_ave = DF3_3H_ave[(DF3_3H_ave[response_counting] > min)]
DF3_3H_ave.head(4)

<ipython-input-51-bd270ac6fc68>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DF3_3H_ave= DF3.set_index('datetime').resample('3H').mean().round(2)


,responses,feel_air,feel_temp,feel_bright,feel_health,feel_noise,air_smell,air_heavy,air_dry,air_dust,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
datetime,,,,,,,,,,,,,,,,,,,,,
2023-03-13 06:00:00,34,2.47,2.00,1.76,1.91,2.53,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,0.0,0.00,1.00,NaN,1.00
2023-03-14 06:00:00,30,2.77,2.40,2.00,2.50,3.07,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.5,0.0,1.0,0.33,1.00,1.0,0.50
2023-03-14 09:00:00,23,2.74,2.48,1.83,2.30,2.43,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,NaN,NaN,NaN,0.00,1.00,1.0,0.00
2023-03-15 09:00:00,41,2.80,2.45,2.20,2.34,2.98,0.0,1.0,0.5,0.5,...,1.0,1.0,0.0,1.0,0.0,0.0,0.25,0.33,0.5,0.25


In [ ]:
DF3_D_ave= DF3.set_index('datetime').resample('D').mean().round(2)
DF3_D_ave['responses'] = DF3.loc[:,['responses', 'datetime']].set_index('datetime').resample('D').sum().round(2)
DF3_D_ave = DF3_D_ave[(DF3_D_ave[response_counting] > min)]
DF3_D_ave.head(4)

<ipython-input-52-eb1933cbeda4>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DF3_D_ave= DF3.set_index('datetime').resample('D').mean().round(2)


,responses,feel_air,feel_temp,feel_bright,feel_health,feel_noise,air_smell,air_heavy,air_dry,air_dust,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
datetime,,,,,,,,,,,,,,,,,,,,,
2023-03-13,34,2.47,2.00,1.76,1.91,2.53,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,1.0,1.0,0.0,0.00,1.00,NaN,1.00
2023-03-14,53,2.75,2.43,1.92,2.42,2.79,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.5,0.0,1.0,0.25,1.00,1.0,0.38
2023-03-15,41,2.80,2.45,2.20,2.34,2.98,0.0,1.0,0.5,0.5,...,1.0,1.0,0.0,1.0,0.0,0.0,0.25,0.33,0.5,0.25
2023-03-16,25,2.68,2.32,1.84,2.20,2.84,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,1.00,1.0,0.00


In [ ]:
def description(in_df):
    xf = in_df
    xf = xf.reset_index()
    datechange = pd.to_datetime(xf['datetime'])
    xf['date']  = datechange.dt.strftime('%Y-%m-%d')
    xf['time']  = datechange.dt.strftime('%H:%M')
    xf['hour']  = datechange.dt.hour
    daily_check = pd.pivot_table(xf, values=response_counting, index= 'date',aggfunc=np.sum)
    #daily_check.columns = [in_df.dtypes]
    return daily_check
    #xf.columns = xf.info
description(DF3_20m)

,responses
date,
2023-03-13,34
2023-03-14,53
2023-03-15,41
2023-03-16,25
2023-03-17,29
2023-03-18,0
2023-03-19,0
2023-03-20,44
2023-03-21,0


In [22]:
def description(in_df):
    xf = in_df
    xf = xf.reset_index()
    datechange = pd.to_datetime(xf['datetime'])
    xf['date']  = datechange.dt.strftime('%Y-%m-%d')
    xf['time']  = datechange.dt.strftime('%H:%M')
    xf['hour']  = datechange.dt.hour
    daily_check = pd.pivot_table(xf, values=response_counting, index= 'time',aggfunc=np.sum)
    #daily_check.columns = [in_df.dtypes]
    return daily_check
    #xf.columns = xf.info
description(DF3_20m)

NameError: name 'DF3_20m' is not defined

# Øyra

## Identity

In [141]:
# # classroom at 302 and 303
# folder_year = '2023'
# start_date  = '2023-03-13'
# end_date    = '2023-04-18'

# classroom at 303
folder_year = '2024'
start_date  = '2024-04-03'
end_date    = '2024-04-17'

In [142]:
school_name = 'øyra'
manuf_id    = 'niluapp'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_raw   + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'
export_file_directory   = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

In [143]:
input_file_directory

'C:/Users/azimilga/project/data-raw/2024/øyra/niluapp/'

## Data Caller

In [144]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.txt') or f.endswith ('.csv')] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path) # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
DF1 = pd.concat(dfs1, ignore_index=True)

datechange = pd.to_datetime(DF1['timestart'])
DF1['date'] = datechange.dt.strftime('%Y-%m-%d')
DF1['time'] = datechange.dt.strftime('%H:%M')

# Define the timezones (wrong time, and Europe/Oslo)
IS_timezone = pytz.timezone('Africa/Luanda')  #why africa? time recording in IS Data is little bit wrong. it has 1 hour delatation but it doesnt consider DST principle. so, applying Africa Luanda, is a trick of that
gmt_timezone = pytz.timezone('Europe/London')
oslo_timezone = pytz.timezone('Europe/Oslo')

# Adjust the datetime to the Oslo timezone and account for daylight saving time
daylight_save_convert = datechange.dt.tz_localize(IS_timezone).dt.tz_convert(gmt_timezone)

# Extract the adjusted time
DF1['time'] = daylight_save_convert.dt.strftime('%H:%M')

cols_to_move = ['title', 'surveyid', 'sessionid','date', 'time']
DF1          = DF1[ cols_to_move + [ col for col in DF1.columns if col not in cols_to_move ] ].drop(['timestart', 'timeend'], axis=1)

#this is to transform numericvalue, from 0-4 become 0-3. where 0 and 1, will be converted as 0. then 2 - 4 will be substract by 1.
#DF1['numericvalue'] = (DF1['numericvalue'].astype(int).replace({0 : 1}) - 1)

In [145]:
DF1_index = DF1.iloc[:, :5].drop(['surveyid'], axis=1).iloc[0::5,:].reset_index(level=0, drop=True)

**Fore-questions transformation**

Fore-questions is arranged as 4 questions : Air, Temperature, Noise, Body's conditions, and indoor lighting.


In [ ]:
# Pivot the dataframe so that each word becomes a column
pivot_df = DF1.pivot_table(index=DF1.groupby('text').cumcount(), columns='text', values='numericvalue', aggfunc='first')

# Reset the index to make it a proper dataframe
DF1_first = pivot_df.reset_index(drop=True)
DF1_first.info()

**Sub-questions arrangement**

In [147]:
DF1_air = (DF1.iloc[:, 8:13].astype(str)).replace({'False': 0, 'True': 1}).reset_index(level=0, drop=True).astype(float).round(0)
DF1_temp = (DF1.iloc[:, 13:18].astype(str)).replace({'False': 0, 'True': 1})
DF1_bright = (DF1.iloc[:, 18:21].astype(str)).replace({'False': 0, 'True': 1})
DF1_health = (DF1.iloc[:, 21:25].astype(str)).replace({'False': 0, 'True': 1})

#set how many delays you want to each variable
delay_temp = 1
delay_bright = 1 + delay_temp
delay_health = 1 + delay_bright

#set new columns for delayed input variables
DF1_tempD   = DF1_temp.iloc[delay_temp:].reset_index(drop=True).astype(float).round(0)
DF1_brightD = DF1_bright.iloc[delay_bright:].reset_index(drop=True).astype(float).round(0)
DF1_healthD = DF1_health.iloc[delay_health:].reset_index(drop=True).astype(float).round(0)

DF1_back = pd.concat([DF1_air, DF1_tempD, DF1_brightD, DF1_healthD], axis=1).iloc[0::5,:].reset_index(level=0, drop=True) #.astype(float)

#Special Action
#Cold and Warm Question has answer's range from 1 to 7. but 0 is the empty answer. we have to convert 0 as 0, but 1 to 7 become -3 to 3. note that scale-4 (means not cold nor warm) as 0.
#DF1_back['KALDT ELLER VARMT'] = DF1_back['KALDT ELLER VARMT'].astype(int).apply(lambda x: x - 4 if x > 0 else x)

**Table Assembly**

In [148]:
DF1_new = pd.concat([DF1_index, DF1_first, DF1_back], axis=1) .sort_values(['date', 'time'], ascending=True).reset_index(drop=True)
DF1_new.rename(columns=all_columns_set, inplace=True)
DF1_new['title'].unique()

array(['Øyra klasserom 303'], dtype=object)

## Classrooms

### Klasserom 302

In [231]:
room_id = 302
class_dir = '302'
is_data_class_name = 'Øyra klasserom 302-7b'
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

### Klasserom 303

In [149]:
room_id = 303
class_dir = '303'
is_data_class_name = 'Øyra klasserom 303'
# is_data_class_name = [f for f in os.listdir() if f.startswith('Øyra klasserom 303') ]
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

### Klasserom 339

In [237]:
room_id = 339
class_dir = '339'
is_data_class_name = 'Bratteberg klasserom '+class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

## Reviewer

In [38]:
#testing  = ((DF2['feel_air'] + DF2['feel_temp']+DF2['feel_bright']+DF2['feel_health'] +DF2['feel_noise']))
#testing2 = ((DF2_listed['feel_air'] + DF2_listed['feel_temp']+DF2_listed['feel_bright']+DF2_listed['feel_health'] +DF2_listed['feel_noise']))
#
print ('all response        :', DF2[time_name].count() )
print ('clean response      :', DF2_listed[time_name].count())
print ('unanswered response : ', DF2[time_name].count() - DF2_listed[time_name].count())

all response        : 0
clean response      : 0
unanswered response :  0


In [39]:
DF3 = DF2_listed
DF3['datetime'] = pd.to_datetime(DF3['date'] + ' ' + DF3['time'])
DF3[response_counting] = 1
DF3.insert(8, response_counting, DF3.pop(response_counting))
DF3_1m= DF3.set_index('datetime').resample('1min').sum().round(1)
DF3_15m= DF3.set_index('datetime').resample('15min').sum().round(1)
DF3_20m= DF3.set_index('datetime').resample('15min').sum().round(1)
DF3_30m= DF3.set_index('datetime').resample('30min').sum().round(1)
DF3_H= DF3.set_index('datetime').resample('H').sum().round(1)


In [40]:
DF3_3H_ave= DF3.set_index('datetime').resample('3H').mean().round(2)
DF3_3H_ave['responses'] = DF3.loc[:,['responses', 'datetime']].set_index('datetime').resample('3H').sum().round(2)
DF3_3H_ave = DF3_3H_ave[(DF3_3H_ave[response_counting] > min)]
DF3_3H_ave.head(4)

,school_id,room_id,date,time,manuf_id,instr_id,serial_id,sessionid,responses,feel_air,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
datetime,,,,,,,,,,,,,,,,,,,,,


In [41]:
DF3_D_ave= DF3.set_index('datetime').resample('D').mean().round(2)
DF3_D_ave['responses'] = DF3.loc[:,['responses', 'datetime']].set_index('datetime').resample('D').sum().round(2)
DF3_D_ave = DF3_D_ave[(DF3_D_ave[response_counting] > min)]
DF3_D_ave.head(4)

,school_id,room_id,date,time,manuf_id,instr_id,serial_id,sessionid,responses,feel_air,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
datetime,,,,,,,,,,,,,,,,,,,,,


In [42]:
def description(in_df):
    xf = in_df
    xf = xf.reset_index()
    datechange = pd.to_datetime(xf['datetime'])
    xf['date']  = datechange.dt.strftime('%Y-%m-%d')
    xf['time']  = datechange.dt.strftime('%H:%M')
    xf['hour']  = datechange.dt.hour
    daily_check = pd.pivot_table(xf, values=response_counting, index= 'date',aggfunc=np.sum)
    #daily_check.columns = [in_df.dtypes]
    return daily_check
    #xf.columns = xf.info
description(DF3_20m)

""
date


In [43]:
def description(in_df):
    xf = in_df
    xf = xf.reset_index()
    datechange = pd.to_datetime(xf['datetime'])
    xf['date']  = datechange.dt.strftime('%Y-%m-%d')
    xf['time']  = datechange.dt.strftime('%H:%M')
    xf['hour']  = datechange.dt.hour
    daily_check = pd.pivot_table(xf, values=response_counting, index= 'time',aggfunc=np.sum)
    #daily_check.columns = [in_df.dtypes]
    return daily_check
    #xf.columns = xf.info
description(DF3_20m)

""
time


# Kuben

## Identity

In [86]:
school_name = 'kuben'
manuf_id    = 'niluapp'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_raw   + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'
export_file_directory   = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'
start_date = '2023-11-06'
end_date   = '2023-11-30'

## Data Caller

In [87]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.txt')] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path) # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
DF1 = pd.concat(dfs1, ignore_index=True)

datechange = pd.to_datetime(DF1['timestart'])
DF1['date'] = datechange.dt.strftime('%Y-%m-%d')
DF1['time'] = datechange.dt.strftime('%H:%M')

# Define the timezones (wrong time, and Europe/Oslo)
IS_timezone = pytz.timezone('Africa/Luanda')  #why africa? time recording in IS Data is little bit wrong. it has 1 hour delatation but it doesnt consider DST principle. so, applying Africa Luanda, is a trick of that
gmt_timezone = pytz.timezone('Europe/London')
oslo_timezone = pytz.timezone('Europe/Oslo')

# Adjust the datetime to the Oslo timezone and account for daylight saving time
daylight_save_convert = datechange.dt.tz_localize(IS_timezone).dt.tz_convert(gmt_timezone)

# Extract the adjusted time
DF1['time'] = daylight_save_convert.dt.strftime('%H:%M')

cols_to_move = ['title', 'surveyid', 'sessionid','date', 'time']
DF1          = DF1[ cols_to_move + [ col for col in DF1.columns if col not in cols_to_move ] ].drop(['timestart', 'timeend'], axis=1)

#this is to transform numericvalue, from 0-4 become 0-3. where 0 and 1, will be converted as 0. then 2 - 4 will be substract by 1.
#DF1['numericvalue'] = (DF1['numericvalue'].astype(int).replace({0 : 1}) - 1)

In [88]:
DF1_index = DF1.iloc[:, :5].drop(['surveyid'], axis=1).iloc[0::5,:].reset_index(level=0, drop=True)

**Fore-questions transformation**

Fore-questions is arranged as 4 questions : Air, Temperature, Noise, Body's conditions, and indoor lighting.


In [89]:
# Pivot the dataframe so that each word becomes a column
pivot_df = DF1.pivot_table(index=DF1.groupby('text').cumcount(), columns='text', values='numericvalue', aggfunc='first')

# Reset the index to make it a proper dataframe
DF1_first = pivot_df.reset_index(drop=True)
DF1_first.info()

**Sub-questions arrangement**

In [90]:
DF1_air = (DF1.iloc[:, 8:13].astype(str)).replace({'False': 0, 'True': 1}).reset_index(level=0, drop=True).astype(float).round(0)
DF1_temp = (DF1.iloc[:, 13:18].astype(str)).replace({'False': 0, 'True': 1})
DF1_bright = (DF1.iloc[:, 18:21].astype(str)).replace({'False': 0, 'True': 1})
DF1_health = (DF1.iloc[:, 21:25].astype(str)).replace({'False': 0, 'True': 1})

#set how many delays you want to each variable
delay_temp = 1
delay_bright = 1 + delay_temp
delay_health = 1 + delay_bright

#set new columns for delayed input variables
DF1_tempD   = DF1_temp.iloc[delay_temp:].reset_index(drop=True).astype(float).round(0)
DF1_brightD = DF1_bright.iloc[delay_bright:].reset_index(drop=True).astype(float).round(0)
DF1_healthD = DF1_health.iloc[delay_health:].reset_index(drop=True).astype(float).round(0)

DF1_back = pd.concat([DF1_air, DF1_tempD, DF1_brightD, DF1_healthD], axis=1).iloc[0::5,:].reset_index(level=0, drop=True) #.astype(float)

#Special Action
#Cold and Warm Question has answer's range from 1 to 7. but 0 is the empty answer. we have to convert 0 as 0, but 1 to 7 become -3 to 3. note that scale-4 (means not cold nor warm) as 0.
#DF1_back['KALDT ELLER VARMT'] = DF1_back['KALDT ELLER VARMT'].astype(int).apply(lambda x: x - 4 if x > 0 else x)

**Table Assembly**

In [91]:
DF1_new = pd.concat([DF1_index, DF1_first, DF1_back], axis=1) .sort_values(['date', 'time'], ascending=True).reset_index(drop=True)
DF1_new.rename(columns=all_columns_set, inplace=True)
DF1_new['title'].unique()

array(['Bikuben B3708', 'Bikuben B2708'], dtype=object)

## Classrooms

### Klasserom 2708

In [92]:
room_id = 2708
class_dir = '2708'
is_data_class_name = 'Bikuben B' + class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

In [93]:
DF2 = DF1_new[DF1_new['title']==is_data_class_name].reset_index(drop=True)

DF2 [school_col_name]  = school_name
DF2 [room_col_name]    = class_dir
DF2 [manuf_col_name]   = manuf_id
DF2 [instr_col_name]   = instr_id
DF2 [serial_col_name]  = serial_id

cols_to_move = [school_col_name, room_col_name,'date', 'time', manuf_col_name, instr_col_name, serial_col_name, 'sessionid']
DF2         = DF2[ cols_to_move + [ col for col in DF2.columns if col not in cols_to_move ] ].drop([ 'title'], axis=1)
#DF2['time'].unique()

#filtering rows data regarding to desired specific time. We neglect data if there is a student did not answer anything. for example, from 808 responses, become 757 responses.
min = 0

#we filterize table with new name of DataFrame, and use specific date and time intervals like above. index has been reset as well.
#his tends to eliminate "Lazy students" who dont answers all main questions at once. they were considered as outliers.
DF2_listed = DF2[(DF2['feel_air'] > min) | (DF2['feel_temp']> min) | (DF2['feel_bright']> min) | (DF2['feel_health'] > min) | (DF2['feel_noise'] > min)].reset_index(level=0, drop=True)

#activate this command if we want to select specified date / today's date
#DF2_listed = DF2_listed[DF2_listed['date']==spec_date]
DF2_listed  = DF2_listed[DF2_listed['date'].between(start_date,end_date)]

In [94]:
#put spesific Directory and File Name.
name_time = datetime.strptime((DF2_listed[time_name].iloc[0]),'%H:%M').strftime('%H%M')
name_date = datetime.strptime((DF2_listed[date_name].iloc[0]),'%Y-%m-%d').strftime('%Y%m%d')
directory_path = export_file_directory +class_dir+'/'
filename = school_name+'_'+class_dir+ '_'+ instr_id + '_'+serial_id +'_'+name_date +'_'+ name_time + '.csv'

# save dataframe (accumulative format) to CSV file in specified directory
DF2_listed.to_csv(directory_path + filename, index= False)

### Klasserom 3707

In [ ]:
room_id = 3707
class_dir = '3707'
is_data_class_name = 'Bikuben B' + class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

In [ ]:
DF2 = DF1_new[DF1_new['title']==is_data_class_name].reset_index(drop=True)

DF2 [school_col_name]  = school_name
DF2 [room_col_name]    = class_dir
DF2 [manuf_col_name]   = manuf_id
DF2 [instr_col_name]   = instr_id
DF2 [serial_col_name]  = serial_id

cols_to_move = [school_col_name, room_col_name,'date', 'time', manuf_col_name, instr_col_name, serial_col_name, 'sessionid']
DF2         = DF2[ cols_to_move + [ col for col in DF2.columns if col not in cols_to_move ] ].drop([ 'title'], axis=1)
#DF2['time'].unique()

#filtering rows data regarding to desired specific time. We neglect data if there is a student did not answer anything. for example, from 808 responses, become 757 responses.
min = 0

#we filterize table with new name of DataFrame, and use specific date and time intervals like above. index has been reset as well.
#his tends to eliminate "Lazy students" who dont answers all main questions at once. they were considered as outliers.
DF2_listed = DF2[(DF2['feel_air'] > min) | (DF2['feel_temp']> min) | (DF2['feel_bright']> min) | (DF2['feel_health'] > min) | (DF2['feel_noise'] > min)].reset_index(level=0, drop=True)

#activate this command if we want to select specified date / today's date
#DF2_listed = DF2_listed[DF2_listed['date']==spec_date]
DF2_listed  = DF2_listed[DF2_listed['date'].between(start_date,end_date)]

In [ ]:
#put spesific Directory and File Name.
name_time = datetime.strptime((DF2_listed['time'].iloc[0]),'%H:%M').strftime('%H%M')
name_date = datetime.strptime((DF2_listed['date'].iloc[0]),'%Y-%m-%d').strftime('%Y%m%d')
directory_path = export_file_directory +class_dir+'/'
filename = school_name+'_'+class_dir+ '_'+ instr_id + '_'+serial_id +'_'+name_date +'_'+ name_time + '.csv'

# save dataframe (accumulative format) to CSV file in specified directory
DF2_listed.to_csv(directory_path + filename, index= False)

### Klasserom 3708

In [99]:
room_id = 3708
class_dir = '3708'
is_data_class_name = 'Bikuben B' + class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

In [100]:
DF2 = DF1_new[DF1_new['title']==is_data_class_name].reset_index(drop=True)

DF2 [school_col_name]  = school_name
DF2 [room_col_name]    = class_dir
DF2 [manuf_col_name]   = manuf_id
DF2 [instr_col_name]   = instr_id
DF2 [serial_col_name]  = serial_id

cols_to_move = [school_col_name, room_col_name,'date', 'time', manuf_col_name, instr_col_name, serial_col_name, 'sessionid']
DF2         = DF2[ cols_to_move + [ col for col in DF2.columns if col not in cols_to_move ] ].drop([ 'title'], axis=1)
#DF2['time'].unique()

#filtering rows data regarding to desired specific time. We neglect data if there is a student did not answer anything. for example, from 808 responses, become 757 responses.
min = 0

#we filterize table with new name of DataFrame, and use specific date and time intervals like above. index has been reset as well.
#his tends to eliminate "Lazy students" who dont answers all main questions at once. they were considered as outliers.
DF2_listed = DF2[(DF2['feel_air'] > min) | (DF2['feel_temp']> min) | (DF2['feel_bright']> min) | (DF2['feel_health'] > min) | (DF2['feel_noise'] > min)].reset_index(level=0, drop=True)

#activate this command if we want to select specified date / today's date
#DF2_listed = DF2_listed[DF2_listed['date']==spec_date]
DF2_listed  = DF2_listed[DF2_listed['date'].between(start_date,end_date)]

In [101]:
#put spesific Directory and File Name.
name_time = datetime.strptime((DF2_listed[time_name].iloc[0]),'%H:%M').strftime('%H%M')
name_date = datetime.strptime((DF2_listed[date_name].iloc[0]),'%Y-%m-%d').strftime('%Y%m%d')

directory_path = export_file_directory +class_dir+'/'
filename = school_name+'_'+class_dir+ '_'+ instr_id + '_'+serial_id +'_'+name_date +'_'+ name_time + '.csv'

# save dataframe (accumulative format) to CSV file in specified directory
DF2_listed.to_csv(directory_path + filename, index= False)

## Reviewer

In [ ]:
DF2_listed

,school_id,room_id,date,time,manuf_id,instr_id,serial_id,sessionid,responses,feel_air,...,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin,datetime
0,kuben,3708,2023-04-18,12:10,niluapp,is_data,noserial,6752a9a2-d977-4ecc-90af-ac156627ea63,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:10:00
1,kuben,3708,2023-04-18,12:10,niluapp,is_data,noserial,8a0fc60a-4943-4498-b478-c9ede3a5eb50,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:10:00
2,kuben,3708,2023-04-18,12:10,niluapp,is_data,noserial,b3a92043-cc0d-483c-a62b-f9929b212dd5,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:10:00
3,kuben,3708,2023-04-18,12:11,niluapp,is_data,noserial,019b2a27-143f-40b7-978e-4a60e4bf3fc8,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:11:00
4,kuben,3708,2023-04-18,12:11,niluapp,is_data,noserial,0afb8591-a0e5-4e42-abfc-3e2707d0b12e,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:11:00
5,kuben,3708,2023-04-18,12:11,niluapp,is_data,noserial,2f3c3b71-e174-4544-9eea-fb01e71e9eda,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:11:00
6,kuben,3708,2023-04-18,12:11,niluapp,is_data,noserial,46d74cb8-0954-46a0-bb38-6ca09ff151c1,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:11:00
7,kuben,3708,2023-04-18,12:11,niluapp,is_data,noserial,795a3f9d-3017-46e8-8854-a130fe5da13c,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:11:00
8,kuben,3708,2023-04-18,12:11,niluapp,is_data,noserial,7cd03cf9-1a9e-4cda-847c-ba41ac99cee7,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:11:00
9,kuben,3708,2023-04-18,12:11,niluapp,is_data,noserial,9a99f667-d17e-481d-92b0-756810823a25,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-18 12:11:00


In [ ]:
#testing  = ((DF2['feel_air'] + DF2['feel_temp']+DF2['feel_bright']+DF2['feel_health'] +DF2['feel_noise']))
#testing2 = ((DF2_listed['feel_air'] + DF2_listed['feel_temp']+DF2_listed['feel_bright']+DF2_listed['feel_health'] +DF2_listed['feel_noise']))
#
print ('all response        :', DF2[time_name].count() )
print ('clean response      :', DF2_listed[time_name].count())
print ('unanswered response : ', DF2[time_name].count() - DF2_listed[time_name].count())

all response        : 26
clean response      : 23
unanswered response :  3


In [ ]:
DF3 = DF2_listed
DF3['datetime'] = pd.to_datetime(DF3['date'] + ' ' + DF3['time'])
DF3[response_counting] = 1
DF3.insert(8, response_counting, DF3.pop(response_counting))
DF3_1m= DF3.set_index('datetime').resample('1min').sum().round(1)
DF3_15m= DF3.set_index('datetime').resample('15min').sum().round(1)
DF3_20m= DF3.set_index('datetime').resample('15min').sum().round(1)
DF3_30m= DF3.set_index('datetime').resample('30min').sum().round(1)
DF3_H= DF3.set_index('datetime').resample('H').sum().round(1)


<ipython-input-630-d6ddd607386e>:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DF3_1m= DF3.set_index('datetime').resample('1min').sum().round(1)
<ipython-input-630-d6ddd607386e>:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DF3_15m= DF3.set_index('datetime').resample('15min').sum().round(1)
<ipython-input-630-d6ddd607386e>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DF3_20m= DF3.set_index('datetime').resample(

In [ ]:
DF3_3H_ave= DF3.set_index('datetime').resample('3H').mean().round(2)
DF3_3H_ave['responses'] = DF3.loc[:,['responses', 'datetime']].set_index('datetime').resample('3H').sum().round(2)
DF3_3H_ave = DF3_3H_ave[(DF3_3H_ave[response_counting] > min)]
DF3_3H_ave.head(4)

<ipython-input-631-bd270ac6fc68>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DF3_3H_ave= DF3.set_index('datetime').resample('3H').mean().round(2)


,responses,feel_air,feel_temp,feel_bright,feel_health,feel_noise,air_smell,air_heavy,air_dry,air_dust,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
datetime,,,,,,,,,,,,,,,,,,,,,
2023-04-18 12:00:00,22,2.5,2.18,2.09,2.45,2.14,0.5,1.0,1.0,0.5,...,0.5,1.0,0.5,NaN,NaN,NaN,0.67,0.33,0.67,0.67
2023-04-19 06:00:00,1,4.0,4.00,3.00,4.00,2.00,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,NaN,NaN,NaN,1.00,0.00,1.00,1.00


In [ ]:
DF3_D_ave= DF3.set_index('datetime').resample('D').mean().round(2)
DF3_D_ave['responses'] = DF3.loc[:,['responses', 'datetime']].set_index('datetime').resample('D').sum().round(2)
DF3_D_ave = DF3_D_ave[(DF3_D_ave[response_counting] > min)]
DF3_D_ave.head(4)

<ipython-input-632-eb1933cbeda4>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DF3_D_ave= DF3.set_index('datetime').resample('D').mean().round(2)


,responses,feel_air,feel_temp,feel_bright,feel_health,feel_noise,air_smell,air_heavy,air_dry,air_dust,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
datetime,,,,,,,,,,,,,,,,,,,,,
2023-04-18,22,2.5,2.18,2.09,2.45,2.14,0.5,1.0,1.0,0.5,...,0.5,1.0,0.5,NaN,NaN,NaN,0.67,0.33,0.67,0.67
2023-04-19,1,4.0,4.00,3.00,4.00,2.00,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,NaN,NaN,NaN,1.00,0.00,1.00,1.00


In [ ]:
def description(in_df):
    xf = in_df
    xf = xf.reset_index()
    datechange = pd.to_datetime(xf['datetime'])
    xf['date']  = datechange.dt.strftime('%Y-%m-%d')
    xf['time']  = datechange.dt.strftime('%H:%M')
    xf['hour']  = datechange.dt.hour
    daily_check = pd.pivot_table(xf, values=response_counting, index= 'date',aggfunc=np.sum)
    #daily_check.columns = [in_df.dtypes]
    return daily_check
    #xf.columns = xf.info
description(DF3_20m)

,responses
date,
2023-04-18,22
2023-04-19,1


In [ ]:
def description(in_df):
    xf = in_df
    xf = xf.reset_index()
    datechange = pd.to_datetime(xf['datetime'])
    xf['date']  = datechange.dt.strftime('%Y-%m-%d')
    xf['time']  = datechange.dt.strftime('%H:%M')
    xf['hour']  = datechange.dt.hour
    daily_check = pd.pivot_table(xf, values=response_counting, index= 'time',aggfunc=np.sum)
    #daily_check.columns = [in_df.dtypes]
    return daily_check
    #xf.columns = xf.info
description(DF3_20m)

,responses
time,
00:00,0
00:15,0
00:30,0
00:45,0
01:00,0
...,...
22:45,0
23:00,0
23:15,0


In [ ]:
DF2 = DF1_new[DF1_new['title']==is_data_class_name].reset_index(drop=True)

DF2 [school_col_name]  = school_name
DF2 [room_col_name]    = class_dir
DF2 [manuf_col_name]   = manuf_id
DF2 [instr_col_name]   = instr_id
DF2 [serial_col_name]  = serial_id

cols_to_move = [school_col_name, room_col_name,'date', 'time', manuf_col_name, instr_col_name, serial_col_name, 'sessionid']
DF2         = DF2[ cols_to_move + [ col for col in DF2.columns if col not in cols_to_move ] ].drop([ 'title'], axis=1)
#DF2['time'].unique()

#filtering rows data regarding to desired specific time. We neglect data if there is a student did not answer anything. for example, from 808 responses, become 757 responses.
min = 0

#we filterize table with new name of DataFrame, and use specific date and time intervals like above. index has been reset as well.
#his tends to eliminate "Lazy students" who dont answers all main questions at once. they were considered as outliers.
DF2_listed = DF2[(DF2['feel_air'] > min) | (DF2['feel_temp']> min) | (DF2['feel_bright']> min) | (DF2['feel_health'] > min) | (DF2['feel_noise'] > min)].reset_index(level=0, drop=True)

#activate this command if we want to select specified date / today's date
#DF2_listed = DF2_listed[DF2_listed['date']==spec_date]

In [ ]:
#put spesific Directory and File Name.
name_time =DF2_listed['time'].iloc[0]
name_date =DF2_listed['date'].iloc[0]
directory_path = export_file_directory +class_dir+'/'
filename = school_name+'_'+class_dir+ '_'+ instr_id + '_'+serial_id +'_'+name_date +'_'+ name_time + '.csv'

# save dataframe (accumulative format) to CSV file in specified directory
DF2_listed.to_csv(directory_path + filename, index= False)

# Brannfjell

In [127]:
# # classroom 23 and 27
# folder_year = '2023'
# start_date =  '2023-03-08'
# end_date   =  '2023-04-26'

# # classroom 11 dan 23 and 27
# folder_year = '2023'
# start_date =  '2023-11-11'
# end_date   =  '2023-11-28'

# # classroom 23 and 27
# folder_year = '2024'
# start_date = '2024-02-26'
# end_date   =  '2024-03-08'

# # classroom 11 dan 23 and 27
folder_year = '2024'
start_date = '2024-04-14'
end_date   =  '2024-04-30'

## Identity

In [128]:
school_name = 'brannfjell'
manuf_id    = 'niluapp'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_raw   + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'
export_file_directory   = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'


## Data Caller

In [129]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.txt') or f.endswith('.csv')] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path) # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
DF1 = pd.concat(dfs1, ignore_index=True)

datechange = pd.to_datetime(DF1['timestart'])
DF1['date'] = datechange.dt.strftime('%Y-%m-%d')
DF1['time'] = datechange.dt.strftime('%H:%M')

# Define the timezones (wrong time, and Europe/Oslo)
IS_timezone = pytz.timezone('Africa/Luanda')  #why africa? time recording in IS Data is little bit wrong. it has 1 hour delatation but it doesnt consider DST principle. so, applying Africa Luanda, is a trick of that
gmt_timezone = pytz.timezone('Europe/London')
oslo_timezone = pytz.timezone('Europe/Oslo')

# Adjust the datetime to the Oslo timezone and account for daylight saving time
daylight_save_convert = datechange.dt.tz_localize(IS_timezone).dt.tz_convert(gmt_timezone)

# Extract the adjusted time
DF1['time'] = daylight_save_convert.dt.strftime('%H:%M')

cols_to_move = ['title', 'surveyid', 'sessionid','date', 'time']
DF1          = DF1[ cols_to_move + [ col for col in DF1.columns if col not in cols_to_move ] ].drop(['timestart', 'timeend'], axis=1)

#this is to transform numericvalue, from 0-4 become 0-3. where 0 and 1, will be converted as 0. then 2 - 4 will be substract by 1.
#DF1['numericvalue'] = (DF1['numericvalue'].astype(int).replace({0 : 1}) - 1)

In [130]:
DF1_index = DF1.iloc[:, :5].drop(['surveyid'], axis=1).iloc[0::5,:].reset_index(level=0, drop=True)

In [131]:
print(DF1[DF1['text'] == 'LUFTEN I ROMMET NÅ']['numericvalue'].unique())
print(DF1[DF1['text'] == 'TEMPERATUR I ROMMET NÅ']['numericvalue'].unique())
print(DF1[DF1['text'] == 'LYS I ROMMET NÅ']['numericvalue'].unique())
print(DF1[DF1['text'] == 'HVORDAN ER FORMEN DIN NÅ?']['numericvalue'].unique())

[ 2.  1.  4.  3. nan]
[ 2.  1.  4.  3. nan  5.  6.  7.]
[ 2.  4.  1.  3. nan]
[ 2.  3.  1.  4. nan]


**Fore-questions transformation**

Fore-questions is arranged as 4 questions : Air, Temperature, Noise, Body's conditions, and indoor lighting.


In [132]:
# Pivot the dataframe so that each word becomes a column
pivot_df = DF1.pivot_table(index=DF1.groupby('text').cumcount(), columns='text', values='numericvalue', aggfunc='first')

# Reset the index to make it a proper dataframe
DF1_first = pivot_df.reset_index(drop=True)
DF1_first.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   HVORDAN ER FORMEN DIN NÅ?  221 non-null    float64
 1   LUFTEN I ROMMET NÅ         222 non-null    float64
 2   LYS I ROMMET NÅ            219 non-null    float64
 3   STØY I ROMMET NÅ           218 non-null    float64
 4   TEMPERATUR I ROMMET NÅ     222 non-null    float64
dtypes: float64(5)
memory usage: 8.9 KB


**Sub-questions arrangement**

In [133]:
DF1_air = (DF1.iloc[:, 8:13].astype(str)).replace({'False': 0, 'True': 1}).reset_index(level=0, drop=True).astype(float).round(0)
DF1_temp = (DF1.iloc[:, 13:18].astype(str)).replace({'False': 0, 'True': 1})
DF1_bright = (DF1.iloc[:, 18:21].astype(str)).replace({'False': 0, 'True': 1})
DF1_health = (DF1.iloc[:, 21:25].astype(str)).replace({'False': 0, 'True': 1})

#set how many delays you want to each variable
delay_temp = 1
delay_bright = 1 + delay_temp
delay_health = 1 + delay_bright

#set new columns for delayed input variables
DF1_tempD   = DF1_temp.iloc[delay_temp:].reset_index(drop=True).astype(float).round(0)
DF1_brightD = DF1_bright.iloc[delay_bright:].reset_index(drop=True).astype(float).round(0)
DF1_healthD = DF1_health.iloc[delay_health:].reset_index(drop=True).astype(float).round(0)

DF1_back = pd.concat([DF1_air, DF1_tempD, DF1_brightD, DF1_healthD], axis=1).iloc[0::5,:].reset_index(level=0, drop=True) #.astype(float)

#Special Action
#Cold and Warm Question has answer's range from 1 to 7. but 0 is the empty answer. we have to convert 0 as 0, but 1 to 7 become -3 to 3. note that scale-4 (means not cold nor warm) as 0.
#DF1_back['KALDT ELLER VARMT'] = DF1_back['KALDT ELLER VARMT'].astype(int).apply(lambda x: x - 4 if x > 0 else x)

**Table Assembly**

In [134]:
DF1_new = pd.concat([DF1_index, DF1_first, DF1_back], axis=1) .sort_values(['date', 'time'], ascending=True).reset_index(drop=True)
DF1_new.rename(columns=all_columns_set, inplace=True)
print(DF1_new['title'].unique())
print(DF1_new['date'].unique())

['Brannfjell Rom 23' 'Brannfjell Rom 27' 'Brannfjell Rom 11']
['2024-02-26' '2024-02-28' '2024-02-29' '2024-03-04' '2024-03-06'
 '2024-03-08' '2024-04-15' '2024-04-16' '2024-04-29' '2024-04-30']


## Classrooms

### Klasserom 11

In [135]:
room_id = 11
class_dir = '11'
is_data_class_name = 'Brannfjell Rom ' + class_dir
instr_id = 'is_data'
serial_id = 'noserial'

### Klasserom 23

In [137]:
room_id = 23
class_dir = '23'
is_data_class_name = 'Brannfjell Rom ' + class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

### Klasserom 27

In [139]:
room_id = 27
class_dir = '27'
is_data_class_name = 'Brannfjell Rom ' + class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

### Klasserom 31

In [57]:
room_id = 31
class_dir = '31'
is_data_class_name = 'Brannfjell Rom ' + class_dir
instr_id = 'is_data'
serial_id = 'noserial'
file_name = instr_id+'-latest.csv'

# Processing

In [13]:
DF2 = DF1_new[DF1_new['title'].str.contains(class_dir)].reset_index(drop=True)

DF2 [school_col_name]  = school_name
DF2 [room_col_name]    = class_dir
DF2 [manuf_col_name]   = manuf_id
DF2 [instr_col_name]   = instr_id
DF2 [serial_col_name]  = serial_id

cols_to_move = [school_col_name, room_col_name,'date', 'time', manuf_col_name, instr_col_name, serial_col_name, 'sessionid']
DF2         = DF2[ cols_to_move + [ col for col in DF2.columns if col not in cols_to_move ] ].drop([ 'title'], axis=1)
#DF2['time'].unique()

#filtering rows data regarding to desired specific time. We neglect data if there is a student did not answer anything. for example, from 808 responses, become 757 responses.
min = 0

#we filterize table with new name of DataFrame, and use specific date and time intervals like above. index has been reset as well.
#his tends to eliminate "Lazy students" who dont answers all main questions at once. they were considered as outliers.
DF2_listed = DF2[(DF2['feel_air'] > min) | (DF2['feel_temp']> min) | (DF2['feel_bright']> min) | (DF2['feel_health'] > min) | (DF2['feel_noise'] > min)].reset_index(level=0, drop=True)


#activate this command if we want to select specified date / today's date
#DF2_listed = DF2_listed[DF2_listed['date']==spec_date]
DF2_listed  = DF2_listed[DF2_listed['date'].between(start_date,end_date)]
    
DF2_listed['temp_coldhot'] = DF2_listed.apply(fill_temp_coldhot, axis=1)

#put spesific Directory and File Name.
name_time = datetime.strptime((DF2_listed[time_name].iloc[0]),'%H:%M').strftime('%H%M')
name_date = datetime.strptime((DF2_listed[date_name].iloc[0]),'%Y-%m-%d').strftime('%Y%m%d')

directory_path = export_file_directory +class_dir+'/'
filename       = school_name+'_'+class_dir+ '_'+ instr_id + '_'+serial_id +'_'+name_date +'_'+ name_time + '.csv'

# replace_dict2 = {1:4, 7:4, 6:3, 2:3, 5:2, 3:2, 4:1}
# DF2_listed['feel_temp'] = DF2_listed['feel_temp'] .replace(replace_dict2)

# save dataframe (accumulative format) to CSV file in specified directory
DF2_listed.to_csv(directory_path + filename, index = False)

In [141]:
print(DF2_listed['school_id'].unique())
print(DF2_listed['room_id'].unique())
print(DF2_listed['date'].unique())
print(DF2_listed['feel_temp'].unique())
print(DF2_listed['feel_air'].unique())

['brannfjell']
['27']
['2024-04-15' '2024-04-16' '2024-04-29' '2024-04-30']
[2. 4. 3. 1.]
[1. 4. 2. 3.]


In [16]:
DF2_listed['temp_coldhot'].describe()

count    574.000000
mean       1.567944
std        2.721144
min        0.000000
25%        0.000000
50%        0.000000
75%        2.000000
max        7.000000
Name: temp_coldhot, dtype: float64

# Reviewer

In [130]:
DF2_listed.head(3)

,school_id,room_id,date,time,manuf_id,instr_id,serial_id,sessionid,feel_air,feel_temp,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
0,brannfjell,27,2024-02-26,08:19,niluapp,is_data,noserial,f7cdc57a-f77b-4516-9d85-494e8d3e43c3,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,brannfjell,27,2024-02-26,08:19,niluapp,is_data,noserial,0a510e7a-a3a9-4c51-94eb-c1d14667d0c7,3.0,3.0,...,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
2,brannfjell,27,2024-02-26,08:19,niluapp,is_data,noserial,0308259a-098a-461d-9a51-5219365dfa0b,3.0,2.0,...,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN


In [17]:
DF2_listed.tail(3)

,school_id,room_id,date,time,manuf_id,instr_id,serial_id,sessionid,feel_air,feel_temp,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
196,brannfjell,11,2023-11-23,11:02,niluapp,is_data,noserial,fb7e8632-c7f9-4d3d-b1b9-bfb37ab047c0,2.0,4.0,...,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
197,brannfjell,11,2023-11-23,11:02,niluapp,is_data,noserial,fb6f8be8-a34e-4158-aafc-5561c83cc03a,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,brannfjell,11,2023-11-23,11:02,niluapp,is_data,noserial,fb6f8be8-a34e-4158-aafc-5561c83cc03a,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#testing  = ((DF2['feel_air'] + DF2['feel_temp']+DF2['feel_bright']+DF2['feel_health'] +DF2['feel_noise']))
#testing2 = ((DF2_listed['feel_air'] + DF2_listed['feel_temp']+DF2_listed['feel_bright']+DF2_listed['feel_health'] +DF2_listed['feel_noise']))
#
print ('all response        :', DF2[time_name].count() )
print ('clean response      :', DF2_listed[time_name].count())
print ('unanswered response : ', DF2[time_name].count() - DF2_listed[time_name].count())

all response        : 208
clean response      : 199
unanswered response :  9


In [19]:
DF3 = DF2_listed
DF3['datetime'] = pd.to_datetime(DF3['date'] + ' ' + DF3['time'])
DF3[response_counting] = 1
DF3.insert(8, response_counting, DF3.pop(response_counting))
DF3_1m= DF3.set_index('datetime').resample('1min').sum().round(1)
DF3_15m= DF3.set_index('datetime').resample('15min').sum().round(1)
DF3_20m= DF3.set_index('datetime').resample('15min').sum().round(1)
DF3_30m= DF3.set_index('datetime').resample('30min').sum().round(1)
DF3_H= DF3.set_index('datetime').resample('H').sum().round(1)


In [20]:
DF3_3H_ave= DF3.set_index('datetime').resample('3H').mean().round(2)
DF3_3H_ave['responses'] = DF3.loc[:,['responses', 'datetime']].set_index('datetime').resample('3H').sum().round(2)
DF3_3H_ave = DF3_3H_ave[(DF3_3H_ave[response_counting] > min)]
DF3_3H_ave

,responses,feel_air,feel_temp,feel_bright,feel_health,feel_noise,air_smell,air_heavy,air_dry,air_dust,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
datetime,,,,,,,,,,,,,,,,,,,,,
2023-11-13 06:00:00,40,2.58,2.44,2.11,2.22,2.44,0.75,0.75,1.00,0.75,...,0.80,0.20,0.00,0.67,0.67,0.00,0.0,0.0,1.0,1.0
2023-11-13 09:00:00,18,2.78,2.50,2.06,2.33,2.50,1.00,1.00,0.67,1.00,...,1.00,0.00,0.00,0.00,1.00,0.00,NaN,NaN,NaN,NaN
2023-11-14 06:00:00,5,2.20,2.80,1.60,1.80,2.00,NaN,NaN,NaN,NaN,...,1.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-15 06:00:00,20,2.55,2.70,2.20,2.05,2.60,1.00,1.00,1.00,1.00,...,1.00,0.00,0.00,0.00,1.00,0.00,NaN,NaN,NaN,NaN
2023-11-16 06:00:00,19,2.26,2.68,2.16,1.63,2.37,1.00,1.00,0.00,1.00,...,0.80,0.00,0.00,0.00,1.00,0.00,NaN,NaN,NaN,NaN
2023-11-17 06:00:00,13,2.62,3.08,2.31,2.38,2.62,1.00,1.00,1.00,1.00,...,0.67,0.50,0.33,0.67,1.00,0.50,NaN,NaN,NaN,NaN
2023-11-20 06:00:00,23,2.57,2.43,2.35,2.14,2.59,1.00,1.00,1.00,1.00,...,1.00,0.00,0.00,0.00,0.75,0.00,NaN,NaN,NaN,NaN
2023-11-20 09:00:00,15,2.47,2.80,2.73,2.07,2.73,1.00,1.00,1.00,1.00,...,1.00,0.17,0.17,0.25,1.00,0.00,NaN,NaN,NaN,NaN
2023-11-21 06:00:00,5,2.20,3.20,2.20,2.20,2.80,NaN,NaN,NaN,NaN,...,1.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
DF3_D_ave= DF3.set_index('datetime').resample('D').mean().round(2)
DF3_D_ave['responses'] = DF3.loc[:,['responses', 'datetime']].set_index('datetime').resample('D').sum().round(2)
DF3_D_ave = DF3_D_ave[(DF3_D_ave[response_counting] > min)]
DF3_D_ave

,responses,feel_air,feel_temp,feel_bright,feel_health,feel_noise,air_smell,air_heavy,air_dry,air_dust,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
datetime,,,,,,,,,,,,,,,,,,,,,
2023-11-13,58,2.65,2.46,2.09,2.26,2.46,0.86,0.86,0.86,0.86,...,0.88,0.12,0.00,0.33,0.83,0.00,0.0,0.0,1.0,1.0
2023-11-14,5,2.20,2.80,1.60,1.80,2.00,NaN,NaN,NaN,NaN,...,1.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-15,20,2.55,2.70,2.20,2.05,2.60,1.00,1.00,1.00,1.00,...,1.00,0.00,0.00,0.00,1.00,0.00,NaN,NaN,NaN,NaN
2023-11-16,19,2.26,2.68,2.16,1.63,2.37,1.00,1.00,0.00,1.00,...,0.80,0.00,0.00,0.00,1.00,0.00,NaN,NaN,NaN,NaN
2023-11-17,13,2.62,3.08,2.31,2.38,2.62,1.00,1.00,1.00,1.00,...,0.67,0.50,0.33,0.67,1.00,0.50,NaN,NaN,NaN,NaN
2023-11-20,38,2.53,2.58,2.50,2.11,2.65,1.00,1.00,1.00,1.00,...,1.00,0.08,0.09,0.12,0.88,0.00,NaN,NaN,NaN,NaN
2023-11-21,5,2.20,3.20,2.20,2.20,2.80,NaN,NaN,NaN,NaN,...,1.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-22,18,2.72,3.11,2.33,2.39,2.78,0.75,1.00,1.00,1.00,...,0.88,0.12,0.00,0.00,0.75,0.00,NaN,NaN,NaN,NaN
2023-11-23,23,2.64,3.13,2.65,2.65,2.48,0.80,1.00,0.80,1.00,...,1.00,0.27,0.09,0.62,0.88,0.12,NaN,NaN,NaN,NaN


In [22]:
def description(in_df):
    xf = in_df
    xf = xf.reset_index()
    datechange = pd.to_datetime(xf['datetime'])
    xf['date']  = datechange.dt.strftime('%Y-%m-%d')
    xf['time']  = datechange.dt.strftime('%H:%M')
    xf['hour']  = datechange.dt.hour
    daily_check = pd.pivot_table(xf, values=response_counting, index= 'date',aggfunc=np.sum)
    #daily_check.columns = [in_df.dtypes]
    return daily_check
    #xf.columns = xf.info
description(DF3_20m)

,responses
date,
2023-11-13,58
2023-11-14,5
2023-11-15,20
2023-11-16,19
2023-11-17,13
2023-11-18,0
2023-11-19,0
2023-11-20,38
2023-11-21,5


In [23]:
def description(in_df):
    xf = in_df
    xf = xf.reset_index()
    datechange = pd.to_datetime(xf['datetime'])
    xf['date']  = datechange.dt.strftime('%Y-%m-%d')
    xf['time']  = datechange.dt.strftime('%H:%M')
    xf['hour']  = datechange.dt.hour
    daily_check = pd.pivot_table(xf, values=response_counting, index= 'time',aggfunc=np.sum)
    #daily_check.columns = [in_df.dtypes]
    return daily_check
    #xf.columns = xf.info
description(DF3_H)

,responses
time,
00:00,0
01:00,0
02:00,0
03:00,0
04:00,0
05:00,0
06:00,103
07:00,40
08:00,3


In [24]:
DF2 = DF1_new[DF1_new['title']==is_data_class_name].reset_index(drop=True)

DF2 [school_col_name]  = school_name
DF2 [room_col_name]    = class_dir
DF2 [manuf_col_name]   = manuf_id
DF2 [instr_col_name]   = instr_id
DF2 [serial_col_name]  = serial_id

cols_to_move = [school_col_name, room_col_name,'date', 'time', manuf_col_name, instr_col_name, serial_col_name, 'sessionid']
DF2         = DF2[ cols_to_move + [ col for col in DF2.columns if col not in cols_to_move ] ].drop([ 'title'], axis=1)
#DF2['time'].unique()

#filtering rows data regarding to desired specific time. We neglect data if there is a student did not answer anything. for example, from 808 responses, become 757 responses.
min = 0

#we filterize table with new name of DataFrame, and use specific date and time intervals like above. index has been reset as well.
#his tends to eliminate "Lazy students" who dont answers all main questions at once. they were considered as outliers.
DF2_listed = DF2[(DF2['feel_air'] > min) | (DF2['feel_temp']> min) | (DF2['feel_bright']> min) | (DF2['feel_health'] > min) | (DF2['feel_noise'] > min)].reset_index(level=0, drop=True)

#activate this command if we want to select specified date / today's date
#DF2_listed = DF2_listed[DF2_listed['date']==spec_date]

In [ ]:
#put spesific Directory and File Name.
name_time =DF2_listed['time'].iloc[0]
name_date =DF2_listed['date'].iloc[0]
directory_path = export_file_directory +class_dir+'/'
filename = school_name+'_'+class_dir+ '_'+ instr_id + '_'+serial_id +'_'+name_date +'_'+ name_time + '.csv'

# save dataframe (accumulative format) to CSV file in specified directory
DF2_listed.to_csv(directory_path + filename)

# Comparison

In [20]:
root_folder = export_file_directory

# Initialize an empty list to store DataFrames
dataframes = []

# Walk through the root folder and its sub-folders
for foldername, subfolders, filenames in os.walk(root_folder):
    for filename in filenames:
        # Check if the file is a CSV file
        if filename.endswith('.csv'):
            # Construct the full path to the CSV file
            file_path = os.path.join(foldername, filename)

            # Read the CSV file into a DataFrame and append it to the list
            df = pd.read_csv(file_path)
            dataframes.append(df)
comparing = pd.concat(dataframes)

comparing = comparing.iloc[:,8:]
comparing.iloc[:,:5] = comparing.iloc[:,:5]-4
comparing.head(6)

,feel_air,feel_temp,feel_bright,feel_health,feel_noise,air_smell,air_heavy,air_dry,air_dust,air_electshock,...,temp_coldfloor,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin
0,-3.0,-1.0,-3.0,-3.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.0,-1.0,-2.0,-2.0,-1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.0,-2.0,-3.0,-2.0,-1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-2.0,-2.0,-2.0,-2.0,-1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-2.0,-3.0,-2.0,0.0,-3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0
5,-2.0,-2.0,-2.0,-1.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(root_folder)

/content/drive/MyDrive/Colab Notebooks/Data Pilot - DMS/data-ready/2023/brannfjell/niluapp/


In [ ]:
comparing.columns

Index(['feel_air', 'feel_temp', 'feel_bright', 'feel_health', 'feel_noise',
       'air_smell', 'air_heavy', 'air_dry', 'air_dust', 'air_electshock',
       'temp_coldhot', 'temp_draw', 'temp_coldfloor', 'temp_heatsun',
       'temp_heater', 'bright_sun', 'bright_lamp_hi', 'bright_lamp_low',
       'health_head', 'health_cough', 'health_tired', 'health_dryskin',
       'Unnamed: 0'],
      dtype='object')

In [ ]:
comparing.head(10)

,feel_air,feel_temp,feel_bright,feel_health,feel_noise,air_smell,air_heavy,air_dry,air_dust,air_electshock,...,temp_heatsun,temp_heater,bright_sun,bright_lamp_hi,bright_lamp_low,health_head,health_cough,health_tired,health_dryskin,Unnamed: 0
0,-2.0,-2.0,-3.0,-1.0,-3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2.0,-2.0,-2.0,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.0,-2.0,-2.0,-2.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.0,-2.0,0.0,-2.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
4,-2.0,-2.0,-2.0,-2.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-1.0,-1.0,-2.0,-1.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-1.0,-2.0,-2.0,-2.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-2.0,-1.0,-2.0,-1.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-2.0,-2.0,-2.0,-1.0,-3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-1.0,-2.0,-2.0,-2.0,-3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Heatmap Correlation
#call the DataFrame
#corr = comparing.corr().round(2)
#corr = comparing.drop(['Unnamed: 0'], axis=1).fillna(0).corr() #.round(2)   #
corr = comparing.fillna(0).corr() #.round(2)   #

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(22, 12))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220,10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,cmap="RdYlGn", vmin=-0.75, vmax=0.75, center=0, cbar=True, annot =True,
            square=True, linewidths=.5, annot_kws={"size": 10})
plt.xticks(rotation=90, fontsize=12)

plt.title(school_name +'- Niluapp'+ ' : Correlation', fontsize=15)
plt.yticks(fontsize=12,rotation=0 )